In [6]:
from sklearn import cluster
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

In [7]:
import pyTigerGraph as tg

TG_HOST = "https://halal.i.tgcloud.io"
TG_USERNAME= "tigergraph"
TG_PASSWORD="XXXX"
TG_GRAPH ="halal"
TG_SECRET= "XXX"

conn = tg.TigerGraphConnection(host=TG_HOST, graphname=TG_GRAPH, username=TG_USERNAME, password=TG_PASSWORD)

print(conn.getToken(TG_SECRET, "1000000")) #uses a lifetime of 1,000,000 seconds

('6cooll4so4lr3nf76ac837ls2i8ks3u6', 1619149912, '2021-04-23 03:51:52')


In [13]:
query = "ProductIngredientLink"
resultTG = conn.runInstalledQuery(query)
results = resultTG[0]['@@tupleRecords']
ProdMan = pd.DataFrame(results)
ProdMan['src']=ProdMan['src'].str.lower()
ProdMan['dest']=ProdMan['dest'].str.lower()
ProdMan=ProdMan.dropna()
ProdMan

,src,dest
0,zona keripik tempe soy bean crips,soybean
1,zona keripik tempe soy bean crips,sugar
2,zona keripik tempe soy bean crips,flavor enhancer
3,zona keripik tempe soy bean crips,salt
4,zona keripik tempe soy bean crips,vegetable oil
...,...,...
131335,,avena
131336,,arrowroot powder organic coconut sugar organic...
131337,,apple
131338,,ascorbic acid


In [14]:
gnx = nx.from_pandas_edgelist(ProdMan, source='src',target='dest')


In [15]:
def graph_to_edge_matrix(G):
    """Convert a networkx graph into an edge matrix.
    See https://www.wikiwand.com/en/Incidence_matrix for a good explanation on edge matrices
   
    Parameters
    ----------
    G : networkx graph
    """
    # Initialize edge matrix with zeros
    edge_mat = np.zeros((len(G), len(G)), dtype=int)

    # Loop to set 0 or 1 (diagonal elements are set to 1)
    for node in G:
        for neighbor in G.neighbors(node):
            edge_mat[node][neighbor] = 1
        edge_mat[node][node] = 1

    return edge_mat

In [17]:
dfm = nx.to_pandas_adjacency(gnx, dtype=int)
dfm.head()

,zona keripik tempe soy bean crips,soybean,sugar,flavor enhancer,salt,vegetable oil,water,chili,yeast,noodle bowl,...,flavorsyesjat milk,e250ohditionne,dark chocolate flavour,waterlecithin sunflower,inforuon,bcttctaves,alantic,austriapuree raspberry,concentrate raspberry,arrowroot powder organic coconut sugar organic coconut cream baking soda sea salt
zona keripik tempe soy bean crips,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
soybean,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
sugar,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
flavor enhancer,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
salt,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
mat=dfm.values

mat

In [28]:
k_clusters = 3
results = []

mykmeans = cluster.KMeans(n_clusters=k_clusters, n_init=200)
mykmeans.fit(mat)
labels = mykmeans.labels_
print(labels)
results = pd.DataFrame([dfm.index,labels]).T

[2 0 5 ... 0 0 0]


In [24]:
results.head()

,0,1
0,zona keripik tempe soy bean crips,3
1,soybean,0
2,sugar,2
3,flavor enhancer,4
4,salt,2


In [27]:
results.to_csv("klasters3.csv")